In [0]:
%sql
USE CATALOG catproject_catalog


In [0]:
%sql
USE SCHEMA ecom_bronze

In [0]:
%sql
CREATE TABLE ecom_bronze.inventory_raw (
  inventory_id       STRING,         -- ID duy nhất cho mỗi bản ghi
  product_id         STRING,         -- Mã sản phẩm
  warehouse_id       STRING,         -- Mã kho (nếu doanh nghiệp có nhiều kho)
  quantity_on_hand   INT,            -- Số lượng thực có trong kho
  quantity_reserved  INT,            -- Đã đặt nhưng chưa xuất
  quantity_available INT,            -- Sẵn sàng để bán = on_hand - reserved
  unit               STRING,         -- Đơn vị tính (e.g., pcs, box)
  snapshot_time      TIMESTAMP,      -- Thời điểm chụp trạng thái tồn kho
  source_system      STRING,
  ingestion_time     TIMESTAMP,
  ingestion_year     INT,
  ingestion_month    INT,
  operation_type     STRING,
  record_hash        STRING,
  source_updated_at  TIMESTAMP
)
USING DELTA
PARTITIONED BY (ingestion_year, ingestion_month);

In [0]:
%sql
COMMENT ON COLUMN ecom_bronze.inventory_raw.inventory_id IS 'Mã duy nhất cho bản ghi tồn kho';
COMMENT ON COLUMN ecom_bronze.inventory_raw.product_id IS 'Mã sản phẩm';
COMMENT ON COLUMN ecom_bronze.inventory_raw.warehouse_id IS 'Kho chứa sản phẩm';
COMMENT ON COLUMN ecom_bronze.inventory_raw.quantity_on_hand IS 'Số lượng có thực tế trong kho';
COMMENT ON COLUMN ecom_bronze.inventory_raw.quantity_reserved IS 'Số lượng đã được đặt giữ chỗ';
COMMENT ON COLUMN ecom_bronze.inventory_raw.quantity_available IS 'Số lượng có thể bán = on_hand - reserved';
COMMENT ON COLUMN ecom_bronze.inventory_raw.unit IS 'Đơn vị tính của sản phẩm';
COMMENT ON COLUMN ecom_bronze.inventory_raw.snapshot_time IS 'Thời điểm snapshot trạng thái tồn kho';
COMMENT ON COLUMN ecom_bronze.inventory_raw.source_system IS 'Nguồn dữ liệu ghi nhận';
COMMENT ON COLUMN ecom_bronze.inventory_raw.ingestion_time IS 'Thời điểm đẩy lên warehouse';
COMMENT ON COLUMN ecom_bronze.inventory_raw.operation_type IS 'Loại thao tác ghi nhận: insert, update, delete';
COMMENT ON COLUMN ecom_bronze.inventory_raw.record_hash IS 'Mã hash giúp phát hiện thay đổi';
COMMENT ON COLUMN ecom_bronze.inventory_raw.source_updated_at IS 'Thời điểm cập nhật từ hệ thống nguồn';


In [0]:
%sql
ALTER TABLE catproject_catalog.ecom_bronze.inventory_raw
SET TBLPROPERTIES (delta.enableChangeDataFeed = true);

In [0]:
%sql
ALTER TABLE catproject_catalog.ecom_bronze.inventory_raw ALTER COLUMN inventory_id SET NOT NULL;
ALTER TABLE catproject_catalog.ecom_bronze.inventory_raw ALTER COLUMN product_id SET NOT NULL;
ALTER TABLE catproject_catalog.ecom_bronze.inventory_raw ALTER COLUMN warehouse_id SET NOT NULL;
ALTER TABLE catproject_catalog.ecom_bronze.inventory_raw ALTER COLUMN quantity_on_hand SET NOT NULL;
ALTER TABLE catproject_catalog.ecom_bronze.inventory_raw ALTER COLUMN quantity_reserved SET NOT NULL;
ALTER TABLE catproject_catalog.ecom_bronze.inventory_raw ALTER COLUMN quantity_available SET NOT NULL;
ALTER TABLE catproject_catalog.ecom_bronze.inventory_raw ALTER COLUMN unit SET NOT NULL;
ALTER TABLE catproject_catalog.ecom_bronze.inventory_raw ALTER COLUMN snapshot_time SET NOT NULL;
ALTER TABLE catproject_catalog.ecom_bronze.inventory_raw ALTER COLUMN source_system SET NOT NULL;
ALTER TABLE catproject_catalog.ecom_bronze.inventory_raw ALTER COLUMN ingestion_time SET NOT NULL;
ALTER TABLE catproject_catalog.ecom_bronze.inventory_raw ALTER COLUMN operation_type SET NOT NULL;
ALTER TABLE catproject_catalog.ecom_bronze.inventory_raw ALTER COLUMN record_hash SET NOT NULL;
ALTER TABLE catproject_catalog.ecom_bronze.inventory_raw ALTER COLUMN source_updated_at SET NOT NULL;

ALTER TABLE catproject_catalog.ecom_bronze.inventory_raw
ADD CONSTRAINT chk_snapshot_time CHECK (snapshot_time < current_timestamp());

ALTER TABLE catproject_catalog.ecom_bronze.inventory_raw
ADD CONSTRAINT chk_ingestion_time CHECK (ingestion_time < current_timestamp());

ALTER TABLE catproject_catalog.ecom_bronze.inventory_raw
ADD CONSTRAINT chk_source_updated_at CHECK (source_updated_at < current_timestamp());

ALTER TABLE catproject_catalog.ecom_bronze.inventory_raw
ADD CONSTRAINT chk_quantity_on_hand_nonnegative CHECK (quantity_on_hand >= 0);

ALTER TABLE catproject_catalog.ecom_bronze.inventory_raw
ADD CONSTRAINT chk_quantity_reserved_nonnegative CHECK (quantity_reserved >= 0);

ALTER TABLE catproject_catalog.ecom_bronze.inventory_raw
ADD CONSTRAINT chk_quantity_available_nonnegative CHECK (quantity_available >= 0);

ALTER TABLE catproject_catalog.ecom_bronze.inventory_raw
ADD CONSTRAINT chk_ingestion_year_nonnegative CHECK (ingestion_year >= 0);

ALTER TABLE catproject_catalog.ecom_bronze.inventory_raw
ADD CONSTRAINT chk_ingestion_month_nonnegative CHECK (ingestion_month >= 0);


